# Seeding

`gymnasium.utils.seeding.np_random` 是 Gymnasium 库中用于生成随机数生成器（RNG）的函数。它基于给定的种子值（seed）或随机选择一个种子值来初始化 NumPy 的随机数生成器。这对于实现可重复的实验和控制环境的随机性非常重要。

### 使用场景
- 初始化环境状态：在环境的 `reset()` 方法中调用，以确保每次重置环境时都可以获得一致的随机状态。
- 实验可复现性：通过提供固定的种子值，研究人员可以确保每次运行实验时环境的行为都是相同的。
- 随机动作选择：在需要基于随机策略选择动作时，比如在探索阶段使用随机动作来探索状态空间。

### 参数说明
- `seed` (可选): 用于创建随机数生成器的种子值。如果为 `None`，则自动生成一个随机种子。种子必须是非负整数。

### 实现功能
函数返回一个元组，包含两个元素：
1. 基于 NumPy 的随机数生成器（`np.random.Generator` 实例）。
2. 用于初始化随机数生成器的种子值。如果没有指定种子值，则返回自动生成的种子值。

### 示例代码
下面的代码展示了如何使用 `np_random` 函数初始化一个随机数生成器，并使用它生成随机数。

```python
from gymnasium.utils import seeding
import numpy as np

# 初始化随机数生成器，不指定种子值
rng, seed = seeding.np_random(None)

print(f"使用的种子值: {seed}")

# 使用随机数生成器生成一些随机数
random_numbers = rng.random(5)
print(f"生成的随机数: {random_numbers}")

# 使用指定的种子值初始化随机数生成器
rng, seed = seeding.np_random(42)
print(f"指定种子值初始化的种子值: {seed}")

# 再次生成随机数
random_numbers = rng.random(5)
print(f"使用指定种子值生成的随机数: {random_numbers}")
```

这个示例首先展示了如何不指定种子值来初始化随机数生成器，并生成随机数。然后，它展示了如何通过指定种子值来初始化生成器，以确保可重复性。

# Environment Checking

`gymnasium.utils.env_checker.check_env` 是 Gymnasium 库中的一个工具函数，用于验证环境是否符合 Gymnasium 的API标准。这个函数通过检查环境的观察空间和动作空间是否被正确设置，以及通过调用 `reset()`、`step()` 和 `render()` 方法来确保环境能够按照预期运行。

### 使用场景
- **开发和调试环境时**：在开发新的环境或修改现有环境时，可以使用 `check_env` 来确保环境遵循 Gymnasium 的API规范。
- **持续集成（CI）中**：在项目的 CI 流程中加入环境检查，以自动验证环境是否符合规范。

### 参数说明
- `env`：要检查的 Gym 环境实例。
- `warn`（已忽略）：之前用于控制是否显示特定警告的参数，现在已被忽略。
- `skip_render_check`：是否跳过对 `render` 方法的检查。默认为 `False`。在某些情况下，例如在持续集成（CI）中可能不希望执行渲染操作。
- `skip_close_check`：是否跳过对 `close` 方法的检查。默认为 `False`。在某些情况下，你可能不需要或不想关闭环境。

### 示例代码
下面的示例演示了如何使用 `check_env` 函数来检查一个简单的自定义环境。

```python
import gymnasium
from gymnasium import spaces
from gymnasium.utils.env_checker import check_env

class CustomEnv(gymnasium.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Discrete(2)
        self.action_space = spaces.Discrete(2)

    def reset(self):
        return self.observation_space.sample()

    def step(self, action):
        observation = self.observation_space.sample()
        reward = 1.0
        done = False
        info = {}
        return observation, reward, done, info

    def render(self, mode="human"):
        pass

    def close(self):
        pass

# 创建环境实例
env = CustomEnv()

# 检查环境
check_env(env, skip_render_check=True)

print("环境检查通过！")
```

这个示例创建了一个简单的自定义环境，并使用 `check_env` 来验证它是否符合 Gymnasium 的API标准。我们通过 `skip_render_check=True` 参数跳过了渲染方法的检查。

# Visualization

`gymnasium.utils.play.play` 是 Gymnasium 库中的一个实用函数，允许用户通过键盘操作来直接玩环境。这个函数适用于测试和展示环境，以及直观地了解环境的动作和观察结果。

### 使用场景
- **亲自测试环境**：通过键盘操作来直接控制环境，可以帮助开发者更好地理解环境的动作和反馈。
- **展示环境**：在演示或展示环境时，可以使用此功能来直观地向观众展示环境的特性和挑战。

### 参数说明
- `env`：要玩的环境。
- `transpose`：如果为 True，则转置观察结果。默认为 True。
- `fps`：每秒执行的环境步数的最大值。如果为 None，则使用 `env.metadata["render_fps"]` 或默认的 30 fps。
- `zoom`：放大观察结果，应为正浮点数。
- `callback`：每一步后执行的回调函数。
- `keys_to_action`：从按键到动作的映射。如果为 None，则使用环境提供的默认映射。
- `seed`：重置环境时使用的随机种子。如果为 None，则不使用种子。
- `noop`：未输入键或输入的键组合未知时使用的动作。

### 示例代码
以下代码展示了如何使用 `play` 函数来玩 CarRacing-v2 环境：

```python
import gymnasium as gym
from gymnasium.utils.play import play
import numpy as np

play(gym.make("CarRacing-v2", render_mode="rgb_array"),  
    keys_to_action={
        "w": np.array([0, 0.7, 0]),
        "a": np.array([-1, 0, 0]),
        "s": np.array([0, 0, 1]),
        "d": np.array([1, 0, 0]),
        "wa": np.array([-1, 0.7, 0]),
        "dw": np.array([1, 0.7, 0]),
        "ds": np.array([1, 0, 1]),
        "as": np.array([-1, 0, 1]),
    },
    noop=np.array([0, 0, 0])
)
```

此代码还展示了如何将 `play` 与实时统计图表结合使用，以实时显示过去 150 步的奖励情况：

```python
from gymnasium.utils.play import PlayPlot, play

def callback(obs_t, obs_tp1, action, rew, terminated, truncated, info):
    return [rew,]

plotter = PlayPlot(callback, 150, ["reward"])             
play(gym.make("CartPole-v1"), callback=plotter.callback)  
```

这些示例展示了 `play` 函数的灵活性和实用性，使得用户可以通过直观的方式与 Gymnasium 环境交互。

`gymnasium.utils.play.PlayPlot` 是 Gymnasium 库中的一个工具类，用于在使用 `play()` 函数玩游戏时，创建实时的绘图以可视化任意指标。这个类使用户能够直观地观察到在玩游戏过程中，各种指标如何变化。

### 使用场景
- **性能监控**：在交互式环境测试中实时监控特定指标，例如即时奖励、累积奖励或动作的大小。
- **环境分析**：分析环境的行为反馈，帮助开发者理解环境动态。

### 参数说明
- `callback`：一个计算从环境转换中得到的指标的函数。这个函数接受一系列参数，如前后观察值、执行的动作、收到的奖励等，并返回一个指标列表。
- `horizon_timesteps`：实时绘图使用的时间范围。
- `plot_names`：绘图的标题列表。

### 实现功能
- **实时绘图**：`PlayPlot` 使用 `matplotlib` 实现实时更新的绘图，这些绘图展示了游戏过程中指标的变化情况。
- **自定义指标计算**：用户可以通过定义 `callback` 函数来计算任意的指标，这些指标将被用于绘图。

### 示例代码

以下代码展示了如何使用 `PlayPlot` 来实时绘制玩 `CartPole-v1` 环境时的即时奖励和累积奖励：

```python
import gymnasium as gym
from gymnasium.utils.play import PlayPlot, play

def compute_metrics(obs_t, obs_tp1, action, rew, terminated, truncated, info):
    # 假设我们想实时观察即时奖励和累积奖励
    return [rew, info.get("cumulative_reward", 0)]

# 创建一个绘图器，设置绘图的时间范围和标题
plotter = PlayPlot(compute_metrics, horizon_timesteps=200, plot_names=["Immediate Reward", "Cumulative Reward"])

# 使用 play 函数玩游戏，并传入 plotter.callback 作为回调函数
play(gym.make("CartPole-v1"), callback=plotter.callback)
```

在这个例子中，`compute_metrics` 函数计算了即时奖励和累积奖励，`PlayPlot` 则根据这些计算结果实时更新绘图。这使得用户在交互式地玩游戏的同时，能够直观地看到这些指标的变化。

## callback

`callback` 是 `PlayPlot` 类中的一个方法，它的作用是在每次环境转换时调用，以便收集环境转换数据并将这些数据用于实时绘图。此方法接受当前和下一个时间步的观察值、执行的动作、收到的奖励、环境是否结束或被截断以及环境返回的额外信息作为参数。

### 使用场景
- **实时数据分析**：在使用 `play()` 函数玩游戏时，通过实时绘制用户定义的指标，如奖励或特定状态的变化，来分析和理解环境的行为。
- **环境调试**：帮助开发者或研究者调试和改进他们的环境或代理行为，通过观察实时的反馈来做出调整。

### 参数说明
- `obs_t`：当前时间步的观察值。
- `obs_tp1`：下一个时间步的观察值。
- `action`：在当前时间步执行的动作。
- `rew`：执行动作后获得的奖励。
- `terminated`：表示环境是否已终止的布尔值。
- `truncated`：表示环境是否被截断的布尔值。
- `info`：环境可能提供的额外信息的字典。

### 实现功能
此方法通过调用用户提供的 `callback` 函数来收集每个时间步的数据，并将这些数据添加到实时更新的绘图中。这允许用户在玩游戏的同时观察他们关心的指标如何随时间变化。

### 示例
在之前的 `PlayPlot` 示例中，`callback` 方法是自动被 `play()` 函数调用的，用户不需要直接调用它。用户需要做的是定义一个 `compute_metrics` 函数，`PlayPlot` 将会在每个时间步调用 `callback` 方法，并将 `compute_metrics` 的返回值添加到绘图中。

这种方式使得在进行强化学习实验或游戏交互式测试时，用户可以轻松地集成实时数据可视化功能，增强了环境交互的直观性和分析的深度。

# Environment pickling

在`gymnasium`库中，"Environment pickling"是指将环境对象序列化为一个字节流，以便于保存到文件或通过网络传输，之后可以再将其反序列化回环境对象。这个过程使用了Python的`pickle`模块，`pickle`是Python的标准库之一，用于序列化和反序列化Python对象结构。

在机器学习和强化学习领域，环境（Environment）指的是一个与智能体（Agent）交互的系统或问题设置，智能体在环境中执行操作，环境则根据智能体的行为返回新的状态和奖励。`gymnasium`库提供了一个简单的接口来构建和使用这样的环境，以便于开发和测试强化学习算法。

环境的pickling非常有用，因为它允许：

- **保存和恢复训练会话**：在长时间的训练过程中，能够保存当前环境的状态，并在需要时恢复，这对于调试和实验非常有帮助。
- **分布式训练**：在分布式系统中训练时，可以通过网络发送环境的状态，使得可以在不同的机器上进行模型的训练和评估。
- **复现实验条件**：确保实验的可复现性，通过保存环境的完整状态，其他研究人员可以准确地复现实验条件，验证和比较结果。

然而，需要注意的是，并非所有的`gymnasium`环境都能够被pickle。这取决于环境中使用的特定对象和资源是否支持pickling。例如，如果环境内部使用了无法被pickle的资源（如打开的文件句柄或者与外部系统的连接），那么该环境就无法直接使用pickle进行序列化。解决这类问题可能需要实现特定的方法来保存和恢复这些资源的状态，或者在环境设计时避免使用难以序列化的资源。

`PlayableGame` 类是一个用于将环境包装起来，允许通过键盘输入与环境进行交互的工具。这个类主要用于在强化学习环境中进行交互式测试和演示，使用户能够直接通过按键来控制代理（环境中的角色或对象），观察环境的响应。

### 参数说明
- **`env`**：要进行交互的环境实例。
- **`keys_to_action`**：一个字典，将键盘按钮的元组映射到环境动作的值。元组中的每个元素是对应按键的键码，可以通过 `pygame` 模块的 `pygame.key` 常量来获取。
- **`zoom`**：如果提供，则对环境的渲染进行放大，以提高观察的清晰度。这个参数对于渲染较小的环境图像特别有用，可以使图像更容易被用户观察。

### 方法说明
- **`process_event(event: Event)`**：此方法用于处理 PyGame 事件。特别地，它用于跟踪当前哪些按钮被按下，以及在 PyGame 窗口关闭时退出 `play()` 函数。
    - **`event`**：要处理的事件对象。这个对象包含了事件的所有信息，例如事件类型（如按键按下、松开等）和事件涉及的按键。

### 使用场景
- **环境演示**：在强化学习研究或开发中，使用 `PlayableGame` 类来直观地展示环境的动态和代理的行为。
- **人机交互测试**：通过手动控制代理在环境中的行动，测试和评估环境的设计、响应以及难度设置。
- **教育和展示**：在教学或演示中，通过交互式控制环境来向观众展示强化学习的概念和环境的特点。

### 示例代码
下面是一个简化的示例代码，展示了如何使用 `PlayableGame` 类来播放一个环境：

```python
import gymnasium as gym
from gymnasium.utils.play import PlayableGame
import pygame

# 创建环境
env = gym.make('CartPole-v1')

# 定义键盘操作到动作的映射
keys_to_action = {
    (pygame.K_LEFT,): 0,  # 向左移动
    (pygame.K_RIGHT,): 1  # 向右移动
}

# 创建可玩游戏实例
playable_game = PlayableGame(env, keys_to_action=keys_to_action, zoom=2.0)

# 游戏主循环
while True:
    # 环境渲染
    env.render()
    for event in pygame.event.get():
        playable_game.process_event(event)
```

此代码段创建了一个 `CartPole-v1` 环境的实例，并定义了左右箭头键对应的动作。然后通过 `PlayableGame` 类实现了一个简单的游戏循环，允许用户通过按键来控制杆子的移动，并观察环境的响应。

`EzPickle` 类是 Gymnasium 库中的一个工具类，用于简化 Python 对象的序列化和反序列化过程。这个类允许对象通过它们的构造函数参数被序列化（pickle）和反序列化（unpickle），这对于那些需要保存和恢复环境状态的场景特别有用，比如在强化学习中保存模型的训练进度。

### 参数说明
- **`*args`**：传递给对象构造函数的位置参数。
- **`**kwargs`**：传递给对象构造函数的关键字参数。

### 使用场景
- **环境保存和加载**：在强化学习中，可能需要保存当前环境的状态，以便于后续可以从相同状态继续训练。`EzPickle` 类可以帮助实现这一需求，通过序列化环境对象并在需要时反序列化。
- **C/C++ 包装器**：对于那些包装了 C/C++ 代码的环境（如 MuJoCo 和 Atari），`EzPickle` 可以用来简化这些环境对象的序列化和反序列化过程。

### 示例代码
以下示例展示了如何定义一个使用 `EzPickle` 进行序列化和反序列化的简单类：

```python
import gymnasium.utils.ezpickle as EzPickle

class Dog(EzPickle):
    def __init__(self, furcolor, tailkind="bushy"):
        # 假设 Animal 是 Dog 的父类
        # Animal.__init__(self)
        EzPickle.__init__(self, furcolor, tailkind)
        self.furcolor = furcolor
        self.tailkind = tailkind

# 创建 Dog 实例
dog = Dog("brown", tailkind="short")

# 序列化 Dog 实例
import pickle
serialized_dog = pickle.dumps(dog)

# 反序列化 Dog 实例
deserialized_dog = pickle.loads(serialized_dog)

# 检查反序列化后的 Dog 是否保持了相同的属性
print(deserialized_dog.furcolor, deserialized_dog.tailkind)
```

在这个示例中，`Dog` 类继承了 `EzPickle` 并在构造函数中调用了 `EzPickle` 的构造函数，传入了所有需要被序列化的参数。这样，当 `Dog` 实例被序列化和反序列化时，这些参数会被自动处理，确保反序列化后的对象保持了相同的状态。

# 保存渲染视频 Save Rendering Videos

`gymnasium.utils.save_video.save_video` 函数允许您从渲染帧中提取并保存视频文件。这个功能特别适用于需要记录和分析代理（agent）在环境中的行为过程的情况，例如在强化学习训练过程中。

### 参数说明
- **`frames`**：要组成视频的渲染帧列表。
- **`video_folder`**：存储录像的文件夹路径。
- **`episode_trigger`**：一个接受整数并返回布尔值的函数，用于决定是否在该回合开始录制视频。如果不提供，将不会基于回合来触发视频录制。
- **`step_trigger`**：一个接受整数并返回布尔值的函数，用于决定是否在该步骤开始录制视频。如果不提供，将不会基于步骤来触发视频录制。
- **`video_length`**：录制回合的长度。如果未指定，则记录整个回合。否则，捕获指定长度的视频片段。
- **`name_prefix`**：将被添加到录制文件名前的前缀。
- **`episode_index`**：当前回合的索引。
- **`step_starting_index`**：第一帧的步骤索引。
- **`save_logger`**：是否记录保存视频的进度，对于需要一段时间保存的长视频非常有用，使用 `"bar"` 来启用。
- **`**kwargs`**：传递给 moviepy 的 `ImageSequenceClip` 的 kwargs。您需要指定 `fps` 或 `duration` 其中之一。

### 使用场景
- 记录训练过程中代理的表现，用于后续分析和展示。
- 创建可视化材料，用于报告或演示。
- 分析代理在特定回合或步骤的行为，以便进行调试或优化。

### 示例代码
以下示例展示了如何在 FrozenLake 环境中记录代理的行为，并将视频保存到 "videos" 文件夹中：

```python
import gymnasium as gym
from gymnasium.utils.save_video import save_video

# 创建环境，启用 rgb_array_list 渲染模式
env = gym.make("FrozenLake-v1", render_mode="rgb_array_list")
_ = env.reset()

step_starting_index = 0
episode_index = 0

# 运行环境
for step_index in range(199):
    action = env.action_space.sample()
    _, _, terminated, truncated, _ = env.step(action)

    # 如果回合结束或被截断，则保存视频
    if terminated or truncated:
        save_video(
            frames=env.render(),
            video_folder="videos",
            fps=env.metadata["render_fps"],
            step_starting_index=step_starting_index,
            episode_index=episode_index
        )
        step_starting_index = step_index + 1
        episode_index += 1
        _ = env.reset()

env.close()
```

在这个示例中，我们在每个回合结束时调用 `save_video` 函数，将该回合的所有渲染帧保存为视频文件。我们使用了环境的 `render_fps` 元数据来设置视频的帧率。

`gymnasium.utils.save_video.capped_cubic_video_schedule` 函数用于决定在哪些回合索引处触发视频录制。这是一个预设的触发器，基于回合数的立方根进行采样，旨在在初期密集地记录，随着回合数增加，记录频率逐渐减少。这样做的目的是在早期详细记录代理的学习过程，并在后期减少存储和处理需求。

### 参数说明
- **`episode_id`**：当前回合的编号。

### 返回值
- 返回一个布尔值，指示是否在当前回合触发视频录制。

### 使用场景
- 在强化学习训练中自动记录关键回合的代理表现，无需手动设置触发条件。
- 在训练早期密集记录代理的表现，以观察学习过程和行为的初始变化。
- 减少在训练后期的视频记录频率，以节省存储空间和处理时间。

### 示例代码
假设你已经有了一个环境和相应的视频保存逻辑，以下示例代码展示如何使用 `capped_cubic_video_schedule` 作为录制视频的触发器：

```python
import gymnasium as gym
from gymnasium.utils.save_video import save_video, capped_cubic_video_schedule

env = gym.make("CartPole-v1", render_mode="rgb_array_list")
episode_count = 100  # 假设我们运行100个回合

for episode_id in range(episode_count):
    _ = env.reset()
    frames = []

    while True:
        action = env.action_space.sample()
        _, _, terminated, truncated, _ = env.step(action)
        frames.append(env.render())

        if terminated or truncated:
            if capped_cubic_video_schedule(episode_id):
                save_video(
                    frames=frames,
                    video_folder="videos",
                    episode_trigger=lambda episode_id: capped_cubic_video_schedule(episode_id),
                    episode_index=episode_id
                )
            break

env.close()
```

在这个例子中，`capped_cubic_video_schedule` 函数用于决定是否在每个回合结束时保存视频。这个函数基于回合编号（`episode_id`）自动计算是否触发视频录制。

# 新旧gym兼容性 Old to New Step API Compatibility

`gymnasium.utils.step_api_compatibility.step_api_compatibility` 函数用于确保环境的 `step()` 方法返回值与期望的 API 兼容。这个函数主要应对 Gymnasium API 更新，其中引入了 `terminated` 和 `truncated` 两个布尔值以替代原来的单一 `done` 布尔值，以提供更详细的环境结束信息。

### 参数说明
- **`step_returns`**：环境 `step()` 方法的返回值，可以是 `(obs, rew, done, info)` 或 `(obs, rew, terminated, truncated, info)` 格式。
- **`output_truncation_bool`**：指示输出是否应返回两个布尔值（新 API）或一个（旧 API）。默认值为 `True`，即使用新 API。
- **`is_vector_env`**：指示 `step_returns` 是否来自向量化环境。

### 返回值
- **`step_returns`**：根据 `output_truncation_bool` 参数，函数返回 `(obs, rew, done, info)` 或 `(obs, rew, terminated, truncated, info)`。

### 使用场景
- 当环境和包装器使用不同的 `step()` API 时，使用该函数可以确保与期望的 API 版本兼容。
- 可以在单个环境或向量化环境中使用，通过 `is_vector_env` 参数区分。

### 示例代码

```python
import gymnasium as gym
from gymnasium.utils.step_api_compatibility import step_api_compatibility

# 单个环境使用旧 API
env = gym.make("CartPole-v0")
_, _ = env.reset()
obs, reward, done, info = step_api_compatibility(env.step(0), output_truncation_bool=False)

# 单个环境使用新 API
obs, reward, terminated, truncated, info = step_api_compatibility(env.step(0), output_truncation_bool=True)

# 向量化环境使用旧 API
vec_env = gym.make_vec("CartPole-v0", vectorization_mode="sync")
_, _ = vec_env.reset()
obs, rewards, dones, infos = step_api_compatibility(vec_env.step([0]), is_vector_env=True, output_truncation_bool=False)

# 向量化环境使用新 API
obs, rewards, terminations, truncations, infos = step_api_compatibility(vec_env.step([0]), is_vector_env=True, output_truncation_bool=True)
```

这个示例展示了如何在单个环境和向量化环境中，根据需要转换 `step()` 方法的返回值格式，以确保与期望的 API 兼容。

`gymnasium.utils.step_api_compatibility.convert_to_terminated_truncated_step_api` 函数用于将环境的 `step()` 方法返回值转换为新的步骤 API，无论输入 API 是什么。新的步骤 API 包括 `(obs, rew, terminated, truncated, info)` 返回格式，其中 `terminated` 和 `truncated` 分别指示环境是否结束以及是否由于最大步数限制而截断。

### 参数说明
- **`step_returns`**：环境 `step()` 方法的返回值，可以是 `(obs, rew, done, info)` 或 `(obs, rew, terminated, truncated, info)` 格式。
- **`is_vector_env`**：指示 `step_returns` 是否来自向量化环境。

### 返回值
- 函数返回 `(obs, rew, terminated, truncated, info)` 格式，符合新的步骤 API。

### 使用场景
此函数适用于需要将现有环境或自定义环境的 `step()` 方法返回值统一转换为新 API 格式的情况。特别是在处理不确定环境返回值格式的场景中非常有用，比如在环境包装器中或在处理多个来源的环境时。

### 示例代码
```python
import gymnasium as gym
from gymnasium.utils.step_api_compatibility import convert_to_terminated_truncated_step_api

# 单个环境示例
env = gym.make("CartPole-v0")
_, _ = env.reset()
step_returns = env.step(0)  # 可能返回旧 API 或新 API 格式
new_api_returns = convert_to_terminated_truncated_step_api(step_returns)
obs, reward, terminated, truncated, info = new_api_returns

# 向量化环境示例
vec_env = gym.make_vec("CartPole-v0", vectorization_mode="sync")
_, _ = vec_env.reset()
step_returns = vec_env.step([0])  # 假设返回旧 API 或新 API 格式
new_api_returns = convert_to_terminated_truncated_step_api(step_returns, is_vector_env=True)
obs, rewards, terminations, truncations, infos = new_api_returns
```

这个示例展示了如何使用 `convert_to_terminated_truncated_step_api` 函数将单个环境或向量化环境的 `step()` 方法返回值转换为新的步骤 API 格式。这确保了无论环境原来使用的是哪种 API，最终处理的都是统一的返回值格式。

`gymnasium.utils.step_api_compatibility.convert_to_done_step_api` 函数用于将环境的 `step()` 方法返回值转换为旧的步骤 API，无论输入 API 是什么。旧的步骤 API 包括 `(obs, rew, done, info)` 返回格式，其中 `done` 指示环境是否结束，无论是因为环境自身的结束条件还是因为达到了最大步数限制。

### 参数说明
- **`step_returns`**：环境 `step()` 方法的返回值，可以是 `(obs, rew, done, info)` 或 `(obs, rew, terminated, truncated, info)` 格式。
- **`is_vector_env`**：指示 `step_returns` 是否来自向量化环境。

### 返回值
- 函数返回 `(obs, rew, done, info)` 格式，符合旧的步骤 API。

### 使用场景
此函数适用于需要将现有环境或自定义环境的 `step()` 方法返回值统一转换为旧 API 格式的情况。特别是在处理旧版代码或框架时非常有用，这些代码或框架可能还没有更新以适应新的 `terminated` 和 `truncated` 标志。

### 示例代码
```python
import gymnasium as gym
from gymnasium.utils.step_api_compatibility import convert_to_done_step_api

# 单个环境示例
env = gym.make("CartPole-v0")
_, _ = env.reset()
step_returns = env.step(0)  # 可能返回旧 API 或新 API 格式
old_api_returns = convert_to_done_step_api(step_returns)
obs, reward, done, info = old_api_returns

# 向量化环境示例
vec_env = gym.make_vec("CartPole-v0", vectorization_mode="sync")
_, _ = vec_env.reset()
step_returns = vec_env.step([0])  # 假设返回旧 API 或新 API 格式
old_api_returns = convert_to_done_step_api(step_returns, is_vector_env=True)
obs, rewards, dones, infos = old_api_returns
```

这个示例展示了如何使用 `convert_to_done_step_api` 函数将单个环境或向量化环境的 `step()` 方法返回值转换为旧的步骤 API 格式。这确保了无论环境原来使用的是哪种 API，最终处理的都是统一的旧 API 格式。